### NEURON KO

In [ ]:
import numpy as np 

In [ ]:
print(np.linspace(15,19,3))

In [ ]:

import sys
sys.path.append('..')  # the parent directory
from AIY_simulation_iclamp import AIY_simulation_iclamp
from g_to_Scm2 import gScm2
import pprint

surf=65.89e-8# surface in cm^2 form neuromorpho AIYL
gAIY=[0.14,1,0.2,0.1,0.92,0.06,0.5,-89.57,1.6]

# leak, slo1iso, kqt1, egl19, slo1egl19, nca, shl1, eleak
    # leak = 0.14
    # slo1iso = 1 
    # kqt1 = 0.2
    # egl19 = 0.1
    # slo1egl19 = 0.92
    # nca = 0.06
    # shl1 = 0.5



# leak
gAIY = [0.14, 0, 0, 0, 0, 0.06, 0, -89.57, 1.6]

# slo1iso
gAIY = [0.14, 1, 0, 0, 0, 0.06, 0, -89.57, 1.6]

# kqt1 
#gAIY = [0.14, 0, 0.2, 0, 0, 0.06, 0, -89.57, 1.6]

# egl19 
#gAIY = [0.14, 0, 0, 0.1, 0, 0.06, 0, -89.57, 1.6]

#slo1egl19
gAIY = [0.14, 0, 0, 0, 0.92, 0.06, 0, -89.57, 1.6]


# shl1
gAIY = [0.14, 0, 0, 0, 0, 0, 0.5, -89.57, 1.6]
 
 



# egl19 + slo1§egl19
# gAIY = [0.14, 0, 0, 0.1, 0.92, 0.06, 0, -89.57, 1.6]

  
        
gAIY_scaled = gScm2(gAIY,surf,6)
pprint.pprint(gAIY_scaled)

        


v, time1, vi_peak, vi = AIY_simulation_iclamp(gAIY_scaled, s1=0.015, s2=0.019, ns=3)
#s2 = 0.035
# best_cc=AIY_simulation_iclamp(gbest,-0.015,0.035,11)
# best_voltage=best_cc[0]
# best_time2=best_cc[1]
# best_VIss=best_cc[3]
# best_VIpeaks=best_cc[2]



# Plotting
import plotly.graph_objects as go

fig = go.Figure()
for i in range(len(v)):
    fig.add_trace(
        go.Scatter(
            x=time1[i], 
            y=v[i], 
            mode='lines', 
            name=f'Run {i + 1}'  # Label each run
        )
    )
fig.update_layout(
    title='Simulation with Only ___ Channel',
    xaxis_title='Time (ms)',
    yaxis_title='Voltage (mV)',
    template='plotly_white' ,
    width = 800,
    height = 400
)
fig.show()


### NeuroML 

In [ ]:
import pandas as pd
from pyneuroml import pynml

def run_simulation_and_get_results(lems_file):
    # Run the simulation with pyNeuroML
    print(f"Running simulation: {lems_file}")
    results = pynml.run_lems_with_jneuroml(
        lems_file, 
        max_memory="1024M", 
        nogui=True, 
        plot=False, 
        load_saved_data=True
    )

    # Extract time and variable values
    time = results["t"]
    voltage_data = {}
    for key in results:
        if key.startswith("pop_AIY") and key.endswith("/v"):  # Filter voltage keys
            compartment = key.split("/")[1]  # Extract compartment index
            voltage_data[f"Run {compartment}"] = results[key]
    
    # Create DataFrame
    df = pd.DataFrame(voltage_data)
    df.insert(0, "Time (ms)", time)  # Add time as the first column
    return df


lems_file = "NeuroML/LEMS_Sim_IClamp_AIY.xml" 
df_AIY_NML = run_simulation_and_get_results(lems_file)
df_AIY_NML = df_AIY_NML * 1000 # Convert voltage from V to mV and time from s to ms
nml_time = df_AIY_NML.iloc[:, 0]
nml_voltages = df_AIY_NML.iloc[:, 1:4]

In [ ]:
nml_voltages.shape

In [ ]:
# Create the Plotly figure
fig = go.Figure()

# Add NEURON simulation data to the plot
fig = go.Figure()

for i in range(len(v)):
    fig.add_trace(
        go.Scatter(
            x=time1[i], 
            y=v[i], 
            mode='lines', 
            name=f'NEURON Run {i + 1}', # Label each run
            line = dict(color = 'blue', width = 2)
        )
    )



# Add NeuroML data to the plot
for col in nml_voltages.columns:
    fig.add_trace(
        go.Scatter(
            x=nml_time,  # Time from NeuroML data
            y=df_AIY_NML[col],  # Voltage from NeuroML data
            mode='lines',
            name=f'NeuroML {col}',  # Label for each trace
            line=dict(color='red', dash='dash', width=1)
        )
    )

# Update layout for clarity
fig.update_layout(
    title='Comparison: NEURON Simulation vs NeuroML Data',
    xaxis_title='Time (ms)',
    yaxis_title='Voltage (mV)',
    template='plotly_white',
    legend=dict(x=0.99, y=0.99),  # Adjust legend position
    height=700,  
    width=800   
)

# Show the plot
fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the size as needed

nml_time = df_AIY_NML.iloc[:, 0]  # Time column
for col in df_AIY_NML.columns[1:]:  # Loop through voltage columns
    ax.plot(
        nml_time, 
        df_AIY_NML[col], 
        label=f"NeuroML Trace {col}", 
        color='red', 
        linestyle='--', 
        linewidth=1
    )


ax.set_title("AIY Neuron Action Potentials: NeuroML", fontsize=14)
ax.set_xlabel("Time (ms)", fontsize=12)
ax.set_ylabel("Voltage (mV)", fontsize=12)


ax.grid(True, linestyle='--', alpha=0.6)
ax.legend(loc="best", fontsize=8)


plt.tight_layout()
plt.show()


In [ ]:
!pynml NeuroML/LEMS_Sim_IClamp_AIY.xml